In [32]:
import csv
import json
import feather # pip install feather-format
import numpy as np
import pandas as pd
import os
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import geopandas as gpd
import ast
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
%matplotlib inline
## Some utility functions
def percentToInt(percentString):
    return int(percentString.rstrip("%"))
def trueFalseToBinary(tfstring):
    return 1 if tfstring=='t' else '0'
def stringListToList(stringList):
    return ast.literal_eval(stringList)
def stringListToCount(stringList):
    return len(ast.literal_eval(stringList))

reviews_data_path = "feather/reviews.feather"
listings_data_path = "feather/listings.feather"
calendar_data_path = "feather/calendar.feather"
sites_data_path = "data/sites_boston.csv"

reviews_data = pd.read_feather(reviews_data_path)
listings_data = pd.read_feather(listings_data_path)
calendar_data = pd.read_feather(calendar_data_path)
sites_data = pd.read_csv(sites_data_path)

# subway data
# https://api-v3.mbta.com/docs/swagger/index.html#/Stop
# holidays data
# https://pypi.org/project/holidays/

In [33]:
features_none_null = ["host_since","host_verifications", 
                        "host_response_rate","host_acceptance_rate",
                        "host_response_time",
                        "host_is_superhost", 
                        "latitude", "longitude",
                        "room_type", "accommodates", 
                        "amenities","minimum_nights",
                        "beds", 
                        "availability_30",	"availability_60",	"availability_90","availability_365", 
                        "number_of_reviews", "review_scores_rating", 
                        "review_scores_accuracy",	"review_scores_cleanliness",
                        "review_scores_checkin",	"review_scores_communication",
                        "review_scores_location",	"review_scores_value",
                        "instant_bookable",  "price",
                        ]
# onehot_vars = ["room_type", ]
# binary_vars = ["host_is_superhost", "instant_bookable" ]

df_none_null = listings_data.copy(deep = True)[features_none_null]
print("before droppign rows with null columns: ", df_none_null.shape)
for col in features_none_null:
    df_none_null = df_none_null[df_none_null[col].notna()]
print("after dropping rows with null columns: ", df_none_null.shape)


before droppign rows with null columns:  (3043, 27)
after dropping rows with null columns:  (1674, 27)


In [ ]:

reference_date = datetime.datetime(2021,7,12)
# adding custom: host_number_of_years
host_number_of_years = [(reference_date - datetime.datetime.strptime(i, '%Y-%m-%d')).days/365
                        for i in df_none_null['host_since']]
df_none_null["host_number_of_years"] = host_number_of_years
df_none_null = df_none_null.drop(['host_since'], axis=1)
df_none_null['price'] = df_none_null['price'].map(lambda x:float(x[1:].replace(',', ''))).astype('int32')
# host reposnse rate; remove % sign, convert to integer
df_none_null['host_response_rate'] = df_none_null['host_response_rate'].map(percentToInt).astype('int32')
#  host_acceptance_rate:
df_none_null['host_acceptance_rate'] = df_none_null['host_acceptance_rate'].map(percentToInt).astype('int32')

host_response_time_dummies = pd.get_dummies(df_none_null['host_response_time'],prefix='host_response_time')
df_none_null = pd.concat([df_none_null,host_response_time_dummies], axis = 1)
df_none_null = df_none_null.drop(['host_response_time'], axis=1)

# host_is_superhost: t/f to binary
df_none_null['host_is_superhost'] = df_none_null['host_is_superhost'].map(trueFalseToBinary).astype('int32')
# "host_verifications",
df_none_null['host_verifications'] = df_none_null['host_verifications'].map(stringListToCount).astype('int32')

room_type_dummies = pd.get_dummies(df_none_null['room_type'],prefix='room_type')
df_none_null = pd.concat([df_none_null,room_type_dummies], axis = 1)
df_none_null = df_none_null.drop(['room_type'], axis=1)

df_none_null['amenities'] = df_none_null['amenities'].map(stringListToCount).astype('int32')
# "availability_30",
df_none_null['availability_30'] = df_none_null['availability_30']/30.0
# "availability_60",
df_none_null['availability_60'] = df_none_null['availability_60']/60.0
# "availability_90",
df_none_null['availability_90'] = df_none_null['availability_90']/90.0
# "availability_365", 
df_none_null['availability_365'] = df_none_null['availability_365']/365.0

df_none_null['instant_bookable'] = df_none_null['instant_bookable'].map(trueFalseToBinary).astype('int32')

print("after processing", df_none_null.shape)
# df_none_null.dtypes

In [ ]:
# Sites features Engineered
